<a href="https://colab.research.google.com/github/EduardaVelascoCatanho/agentesIAnaciberdefesa/blob/main/projetoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip -q install google-genai

In [57]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [58]:

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [59]:
#Instalar o Agent Development Kit

!pip -q install google-adk

In [60]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types #Isso vai criar conteúdo tipo ocorreu na aula 4
from datetime import date
import textwrap #Formatar a saída de texto
from IPython.display import display, Markdown #Exibir texto formatado no Google Colab
import requests #Para requisições HTTP (veja se pode limitar isso a HTTPS)
import warnings

warnings.filterwarnings('ignore')

In [93]:
def call_agent(agent: Agent, message_text: str) -> str:
    #Serviço de sessão em memória
    session_service = InMemorySessionService()
    #Cria nova sessão, permite personalizar ID conforme necessário
    session_id = "session" # Use o mesmo ID aqui
    session = session_service.create_session(app_name=agent.name, user_id="user", session_id=session_id)
    #Cria um runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    #Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    #Itera assincronamente  pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user", session_id=session_id, new_message=content): # Use o mesmo ID aqui
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response


In [79]:
# FUNÇÃO AUXILIAR PARA EXIBIR TEXTO FORMATADO EM MARKDOWN

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [80]:
##########################################
# --- Agente 1:  Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      instruction="""
      Você é um assistente virtual especializado em cibersegurança. Sua principal
      função é fornecer notícias, informações, análises e dicas relevantes sobre
      o cenário da segurança cibernética. Responda de forma clara, concisa e acessível,
      adaptando o nível de detalhe à pergunta do usuário. Mantenha-se atualizado com as
      últimas tendências, ameaças, vulnerabilidades, melhores práticas e ferramentas de
      segurança. Ao fornecer informações, cite suas fontes ou no que você está baseando
      sua resposta.
      Quando estiver referindo-se a um ataque ou tipo de ataque lembre-se de explicitar a natureza
       do ataque, a vulnerabilidade explorada, os principais afetados e quais medidas de contenção
       foram adotadas
       """,
      description="Agente que busca, no Google, informações sobre cibersegurança",
      tools=[google_search]
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [96]:
################################
# --- Agente 2: Farejador --- #
################################

def agente_farejador(topico, documentacao):
  farejador = Agent(
      name="agente_farejador",
      model="gemini-2.0-flash",
      instruction= """ Você é o Agente Farejador, um especialista em segurança de código.
      Sua principal tarefa é analisar os pedaços de código que o usuário te enviar e comparar
      esses códigos com a documentação oficial das tecnologias e bibliotecas que aparecem neles.
      Você vai receber pedaços de código e, com muita atenção, COMPARAR com a documentação. Seu
      dever é encontrar todo e qualquer bug ou trecho malicioso que possa comprometer a segurança
      dos envolvidos. Você fareja falhas que vão além do convencional, como por exemplo práticas que
      a documentação não recomenda.
      Encontrando a falha, você irá explicar qual é o problema encontrado (e fará isso de um jeito claro
      e explicíto), evidênciando onde está a falha.
       Quando possível destaque trechos da documentação oficial.
       Sempre forneça sugestões de como consertar o código, fazer manutenção e/ou melhorar a sintaxe.
      """,
      description="Agente que fareja falhas que possam comprometer a segurança",
      tools=[google_search]
  )

  entrada_do_agente_farejador = f"Tópico: {topico}\nLançamentos buscados: {documentacao}"
  falha = call_agent(farejador, entrada_do_agente_farejador)
  return falha

In [90]:
######################################
# --- Agente 3: Relatório Forense --- #
######################################
def agente_redator(topico, relatorio):
  redator = Agent(
      name="agente_redator",
      model="gemini-2.5-pro-preview-03-25",
      instruction="""
      Você é um agente especializado na elaboração de relatórios forenses de incidentes de
      cibersegurança. Sua tarefa é analisar dados e informações fornecidas sobre um ataque
      cibernético e gerar um relatório completo e objetivo. O relatório deve detalhar todas as
      fases do incidente, o impacto, as evidências coletadas e as recomendações para prevenção futura.

      Instruções Específicas para o Relatório:
      Ao gerar o relatório, inclua as seguintes seções e informações, utilizando sempre uma linguagem clara,
      concisa e profissional:

      1. SUMÁRIO EXECUTIVO --> Forneça um resumo conciso do incidente, incluindo o tipo de ataque, os ativos afetados
      e um escopo geral da situação;
      2. INTRODUÇÃO --> Declare o propósito do relatório e o contexto do incidente, identifique a organização afetada,
      data e hora em que o incidente foi identificado, mencione quem solicitou a investigação e se já é possível mesurar danos,
      sempre diferencie ataque de tentativa de ataque, já indicando sistemas comprometidos (se houver algum);
      3. METODOLOGIA --> Descreva os métodos e ferramentas utilizados na investigação forense, liste as fontes de informação e
      evidências analisadas(logs, tráfego de rede, imagens de disco, etc), explique como/onde/com quem a cadeia de custódia das
      evidências foi mantida (se aplicável);
      4. DETALHES DO INCIDENTE -->
       Linha do tempo: apresente uma cronologia detalhada dos eventos do ataque, desde a detecção até a contenção e erradicação,
       Vetor do ataque: explique como o ataque provavelmente ocorreu (phising, exploração de vulnerabilidade, Man-in-the-Middle, etc)
       Técnicas, táticas e procedimentos (TTPs): descreva o comportamento do atacante durante o incidente, referenciando frameworks (caso
       tenha sido possível reconstruir e correr simulação do evento),
       Malware/Ferramentas utilizadas (se aplicável): identifique todo e qualquer software malicioso ou ferramenta de ataques utilizados,
       inclua nomes, hashes e funcionalidades conhecidas,
       Sistemas e dados comprometidos: liste todos os sistemas, aplicativos e dados que se tem conhecimento de que foram afetados pelo
       ataque. Detalhe também o nível de comprometimento (acesso, modificação, exfiltração...),
       Impacto: avalie o impacto do incidente em termos financeiros, operacionais, de reputação e de conformidade regulatória (por exemplo,
       um ataque de negação de serviço a uma determinada plataforma pode interromper o fluxo de operações e afastar clientes, o que será refletido
       a médio e longo prazo, sendo preciso certo tempo para que a plataforma recupere-se do incidente não só em termos operacionais),
       5. ANÁLISE DE EVIDÊNCIAS --> Analise as evidencias digitais coletadas e explique como elas suportam as conclusões sobre o vetor de ataque, as
       TTPs e os ativos comprometidos, inclua também trechos relevantes de logs (se aplicável), análise de tráfego de rede (se aplicável) ou quaisquer
       outras evidências que sirvam para embasar, de forma concisa, sua análise,
       6. ATRIBUIÇÃO (se aplicável e com ressalvas) --> se houver evidências que aponte para a autoria do ataque (grupo, indivíduo, estado-nação),
       apresente essas evidências (sempre com cautela), reconhecendo as dificuldades da atribuição definitiva, mencionando motivações (pessoais? financeiras?),
       7. RECOMENDAÇÔES --> forneça recomendações específicas e acionáveis para mitigar as vulnerabilidades exploradas, as identificadas e de que maneira
       prevenir futuros incidentes semelhantes. As recomendações devem ser categorizadas (por exemplo: segurança de rede, segurança de endpoints, treinamento de
       usuários, processos de resposta a incidentes) e também devem seguir a metodologia aplicada ao processo de investigação forense,
       8. CONCLUSÃO --> finalização do relatório,
       9. APÊNDICE (quando aplicável) --> fontes e informações adicionais.

       """
  )

  entrada_do_agente_redator = f"Tópico: {topico}\nRelatório: {relatorio}"
  relatorio_final = call_agent(redator, entrada_do_agente_redator)
  return relatorio_final


In [86]:
from google.genai import types

chat_config = types.GenerateContentConfig(
    system_instruction='Você é um assistente pessoal e você sempre responde de forma sucinta'

)
modelo = "gemini-2.0-flash"
chat = client.chats.create(model=modelo, config=chat_config)

In [70]:
# Pergunte ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

#Pergunta genérica
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com o Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

display(Markdown(f"Response:\n {response.text}"))

Response:
 A Imersão IA com Google Gemini mais recente ocorreu de 12 a 16 de maio de 2025. As inscrições para esta edição estavam abertas até 11 de maio de 2025.

A Imersão IA é um curso online e gratuito, realizado em parceria entre o Google e a Alura. O curso é voltado para profissionais de todas as áreas e não exige conhecimento prévio. As aulas ensinam a criar prompts eficazes, automatizar tarefas e desenvolver agentes autônomos. Ao final do curso, os participantes recebem um certificado da Alura e do Google.


# 🕵️‍♂️ Relatório Forense de Incidente de Cibersegurança 🕵️‍♀️

## 🗓️ Data do Incidente: INSERIR DATA E HORA
## 🏢 Organização Afetada: INSERIR NOME DA ORGANIZAÇÃO
## 👤 Investigador Responsável: SEU NOME/NOME DO AGENTE REDATOR

---

## 🎯 Sumário Executivo

> INSERIR UM RESUMO CONCISO DO INCIDENTE. Mencione o tipo de ataque, os principais ativos afetados e o impacto geral em poucas frases. Destaque as conclusões mais importantes e as principais recomendações.

---

## 📜 1. Introdução

> DECLARE O PROPÓSITO DESTE RELATÓRIO E O CONTEXTO DO INCIDENTE. Explique brevemente o que aconteceu e por que esta investigação foi conduzida. Mencione quem solicitou a investigação e a data em que ela começou.

> **Data e Hora da Identificação:** INSERIR DATA E HORA
> **Solicitado Por:** INSERIR QUEM SOLICITOU
> **Danos Iniciais Estimados:** SE JÁ HOUVER UMA ESTIMATIVA
> **Status:** ATAQUE CONFIRMADO / TENTATIVA DE ATAQUE
> **Sistemas Inicialmente Comprometidos (se aplicável):** LISTAR SISTEMAS

---

## ⚙️ 2. Metodologia

> DESCREVA OS MÉTODOS E AS FERRAMENTAS UTILIZADAS NA INVESTIGAÇÃO FORENSE. Liste as principais fontes de informação e evidências analisadas, como logs, tráfego de rede e imagens de disco. Explique brevemente como a cadeia de custódia das evidências foi mantida.

> **Ferramentas Utilizadas:** LISTAR FERRAMENTAS
> **Fontes de Evidência:** LISTAR FONTES (Logs de Servidores, Firewalls, etc.)
> **Cadeia de Custódia:** BREVE DESCRIÇÃO

---

## 💥 3. Detalhes do Incidente

### ⏱️ 3.1 Linha do Tempo dos Eventos

> APRESENTE UMA CRONOLOGIA DETALHADA DOS EVENTOS DO ATAQUE, DESDE A DETECÇÃO ATÉ A CONTENÇÃO E ERRADICAÇÃO. Use uma lista para facilitar a leitura.

> - **DATA E HORA:** [DESCRIÇÃO DO EVENTO]
> - **DATA E HORA:** [DESCRIÇÃO DO EVENTO]
> - ...

### ➡️ 3.2 Vetor do Ataque

> [EXPLIQUE COMO O ATAQUE PROVAVELMENTE OCORREU. Detalhe o método de entrada inicial (phishing, exploração de vulnerabilidade, etc.).]

> **Método de Entrada:** PHISHING / EXPLORAÇÃO DE VULNERABILIDADE / ...
> **Vulnerabilidade Explorada (se conhecida):** CVE-XXXX-XXXX / DESCRIÇÃO

### 🛡️ 3.3 Técnicas, Táticas e Procedimentos (TTPs)

> DESCREVA O COMPORTAMENTO DO ATACANTE DURANTE O INCIDENTE. Se aplicável, mencione as fases do ataque e as técnicas utilizadas, referenciando frameworks como o MITRE ATT&CK.

> **Fase Inicial:** DESCRIÇÃO DAS AÇÕES
> **Movimentação Lateral:** DESCRIÇÃO DAS AÇÕES
> **Escalonamento de Privilégios:** DESCRIÇÃO DAS AÇÕES
> **Persistência:** DESCRIÇÃO DAS AÇÕES
> **Exfiltração de Dados (se aplicável):** DESCRIÇÃO

### 🦠 3.4 Malware e Ferramentas Utilizadas

> IDENTIFIQUE QUALQUER SOFTWARE MALICIOSO OU FERRAMENTA DE ATAQUE UTILIZADA. Inclua nomes, hashes (se disponíveis) e funcionalidades conhecidas.

> **Nome do Malware:** NOME DO MALWARE (se conhecido)
> **Hash (MD5/SHA256):** HASH DO ARQUIVO (se disponível)
> **Funcionalidades:** DESCRIÇÃO DO QUE O MALWARE FAZ
> **Outras Ferramentas:** LISTAR OUTRAS FERRAMENTAS UTILIZADAS

### 💻 3.5 Sistemas e Dados Comprometidos

> LISTE TODOS OS SISTEMAS, APLICATIVOS E TIPOS DE DADOS QUE FORAM AFETADOS PELO ATAQUE. Detalhe o nível de comprometimento para cada um.

> **Servidores:** LISTA DE SERVIDORES E NÍVEL DE COMPROMETIMENTO
> **Estações de Trabalho:** LISTA DE ESTAÇÕES E NÍVEL DE COMPROMETIMENTO
> **Bancos de Dados:** LISTA DE BANCOS E NÍVEL DE COMPROMETIMENTO
> **Tipos de Dados Afetados:** PII, FINANCEIROS, SEGREDOS COMERCIAIS, ...
> **Volume de Dados Afetados (estimativa):** TAMANHO APROXIMADO

### 📉 3.6 Impacto do Incidente

> AVALIE O IMPACTO DO INCIDENTE EM TERMOS FINANCEIROS, OPERACIONAIS, DE REPUTAÇÃO E DE CONFORMIDADE REGULATÓRIA.

> **Impacto Financeiro (estimativa):** VALOR
> **Impacto Operacional:** DESCRIÇÃO DA INTERRUPÇÃO DE SERVIÇOS
> **Impacto na Reputação:** DESCRIÇÃO DOS DANOS À IMAGEM
> **Impacto na Conformidade:** MENÇÃO A POSSÍVEIS VIOLAÇÕES REGULATÓRIAS

---

## 🔍 4. Análise das Evidências

> ANALISE AS EVIDÊNCIAS DIGITAIS COLETADAS E EXPLIQUE COMO ELAS SUSTENTAM AS CONCLUSÕES SOBRE O VETOR DE ATAQUE, AS TTPs E OS ATIVOS COMPROMETIDOS. Inclua trechos relevantes de logs ou outras evidências de forma concisa.

> **Análise de Logs:** RESUMO DAS PRINCIPAIS OBSERVAÇÕES NOS LOGS
> **Análise de Tráfego de Rede:** RESUMO DAS PRINCIPAIS OBSERVAÇÕES NO TRÁFEGO
> **Outras Evidências:** ANÁLISE DE OUTROS ARTEFATOS

---

## 👤 5. Atribuição (se aplicável)

> SE HOUVER EVIDÊNCIAS QUE APONTEM PARA UM POSSÍVEL ATOR DE AMEAÇA, APRESENTE ESSAS EVIDÊNCIAS COM CAUTELA, RECONHECENDO AS DIFICULDADES DA ATRIBUIÇÃO DEFINITIVA. Mencione motivações potenciais.

> **Possível Ator de Ameaça:** NOME DO GRUPO/INDIVÍDUO (SE CONHECIDO)
> **Indicadores de Atribuição:** EVIDÊNCIAS QUE SUGEREM A ATRIBUIÇÃO
> **Motivação Potencial:** FINANCEIRA / POLÍTICA / ...

---

## 🛡️ 6. Recomendações

> FORNEÇA RECOMENDAÇÕES ESPECÍFICAS E ACIONÁVEIS PARA MITIGAR AS VULNERABILIDADES EXPLORADAS E PREVENIR FUTUROS INCIDENTES SEMELHANTES. Categorize as recomendações.

### 🌐 6.1 Segurança de Rede

> - RECOMENDAÇÃO 1
> - RECOMENDAÇÃO 2
> - ...

### 💻 6.2 Segurança de Endpoints

> - RECOMENDAÇÃO 1
> - RECOMENDAÇÃO 2
> - ...

### 🧑‍🏫 6.3 Treinamento de Usuários

> - RECOMENDAÇÃO 1
> - RECOMENDAÇÃO 2
> - ...

### 🚨 6.4 Processos de Resposta a Incidentes

> - RECOMENDAÇÃO 1
> - RECOMENDAÇÃO 2
> - ...

---

##   7. Conclusão

> REITERE AS PRINCIPAIS DESCOBERTAS DO RELATÓRIO E ENFATIZE A IMPORTÂNCIA DAS RECOMENDAÇÕES PARA FORTALECER A POSTURA DE SEGURANÇA DA ORGANIZAÇÃO.

---

## 📚 8. Apêndices (se aplicável)

> INCLUA INFORMAÇÕES ADICIONAIS, COMO LOGS RESUMIDOS, DETALHES TÉCNICOS OU GLOSSÁRIO.

> - ITEM DO APÊNDICE 1
> - ITEM DO APÊNDICE 2
> - ...

In [ ]:
################################################
# --- Inserir lógica do sistema de agentes --- #
################################################
topico = input("Sobre o que gostaria de falar? ")
if not topico:
    print("Você esqueceu de digitar o tema!")
else:
    print(f"Ótimo, hora da diversão analisando o: {topico}")

    data_de_hoje = date.today().strftime("%Y-%m-%d") # Garante que data_de_hoje esteja definida

    lancamentos = agente_buscador(topico, data_de_hoje)
    documentacao = lancamentos
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos))
    print("--------------------------------------------------------------")


    if 'documentacao' in locals(): # Verifica se a variável foi definida
        falha = agente_farejador(topico, documentacao)
        print("\n--- 📝 Resultado do Agente 2 (Farejador) ---\n")
        display(to_markdown(falha))
        print("--------------------------------------------------------------")

        relatorio = falha
        relatorio_final = agente_redator(topico, relatorio)
        print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
        display(to_markdown(relatorio_final))
        print("--------------------------------------------------------------")
    else:
        print("\n--- ⚠️ Aviso: Deu ruim")